<a href="https://colab.research.google.com/github/Giftcomfort/OOU_CSC309_Artificial_Intelligence/blob/main/notebooks/CSC309_Week05_KRR_Logic_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 5 Lab:** Knowledge Representation & Reasoning — Logic & Inference

**Instructor:** Dr Sakinat Folorunso

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> Every code cell is commented line‑by‑line so you can follow the logic precisely.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.  
2. Run the **Setup** cell once.  
3. Work through **Tasks**. Edit only cells marked **`# TODO(Student)`**.  
4. Use **Quick Checks** to test your understanding.  
5. Finish with the **Reflection**. If you finish early, try the **Extensions**.

In [5]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# The '#@param' annotations create form fields in Colab for easy input.

group_members = "Ozakpolor Godgift"  #@param {type:"string"}  # Names of teammates
roles_notes = "Driver/Navigator, decisions, questions"  #@param {type:"string"}  # Short working notes

print("👥 Group:", group_members)        # Echo the group list for confirmation
print("📝 Notes:", roles_notes)          # Echo the notes so they're preserved in output

👥 Group: Ozakpolor Godgift
📝 Notes: Driver/Navigator, decisions, questions


### Learning Objectives
- Encode facts/rules in a small **logic knowledge base**.  
- Query using unification; practice NL→logic, CNF/resolution (by hand).

In [6]:
#@title 🔧 Setup
# We try to use 'kanren' for a taste of relational programming.
# If unavailable, we fall back to a tiny custom rule demo.

import sys, subprocess                                     # Tools for optional installs
def pip_install(pkgs):
    for p in pkgs:
        try: __import__(p.split("==")[0])                  # Try importing
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", p])  # Install if missing

try:
    pip_install(["kanren", "unification", "toolz"])        # Attempt to install kanren ecosystem
    from kanren import Relation, facts, run, var           # Import relational primitives
    using_kanren = True                                     # Flag to indicate the backend in use
except Exception as e:
    print("Using simple fallback (kanren not available):", e) # Inform the student
    using_kanren = False                                    # Switch to fallback
print("✅ Setup complete for Week 5.")

✅ Setup complete for Week 5.


In [7]:
#@title 📐 Starter KB (line‑by‑line comments)

if 'using_kanren' in globals() and using_kanren:    # If kanren is available
    parent = Relation()                              # Declare a binary relation 'parent(x,y)'
    facts(parent, ("ada","sam"), ("sam","tayo"), ("sam","ife"), ("ife","bola"))  # Add some facts
    x = var()                                        # Logic variable 'x'
    y = var()                                        # Logic variable 'y'
    # Query: who are Ada's grandchildren?
    result = run(0, x, parent("ada", x), parent(x, y))   # Find x such that parent(ada,x) and parent(x,y)
    print("Grandchildren of Ada (via kanren):", set(result))  # Show solutions
else:
    # Fallback: a tiny Python-based relation
    PARENT = {("ada","sam"), ("sam","tayo"), ("sam","ife"), ("ife","bola")}  # Parent relation as a set of tuples
    def grandchildren(p):                 # Compute grandchildren of 'p' by relational join
        return {c2 for (p1,c1) in PARENT for (p2,c2) in PARENT if p1==p and p2==c1}
    print("Grandchildren of Ada (fallback):", grandchildren("ada"))  # Show result

Grandchildren of Ada (via kanren): {'sam'}


In [8]:
from kanren import Relation, facts, run, var, lall, eq

# 1. Create relation HUMAN(x)
human = Relation()

# 2. Add fact: Socrates is human
facts(human, ("socrates",))

# 3. Define the rule: All humans are mortal.
# In kanren, we can express this by defining a goal that
# states: if someone is human, then they are considered mortal.
# We infer mortality based on being human.
def is_mortal(person_var):
    """A person is mortal if they are human."""
    return human(person_var)

# Declare a logic variable
x = var()

# 4. Run a query for Socrates to see if he is mortal.
# We are asking: "Who is 'x' such that 'is_mortal(x)' is true AND 'x' is 'socrates'?"
result = run(0, x, lall(is_mortal(x), eq(x, "socrates")))

# Print the result. If 'socrates' is in the result, it means he is inferred to be mortal.
print("Facts and Rule:")
print("- Socrates is human (fact).")
print("- All humans are mortal (rule: if someone is human, they are mortal).")

if "socrates" in result:
    print("\nTherefore, Socrates is mortal (inferred via kanren).")
else:
    print("\nSocrates is not inferred to be mortal.")

Facts and Rule:
- Socrates is human (fact).
- All humans are mortal (rule: if someone is human, they are mortal).

Therefore, Socrates is mortal (inferred via kanren).


### Quick Check
Translate **two** everyday constraints into predicate logic, then back into English.  
E.g., “All 300‑level courses have a CA in Week 7.”


**Constraint 1: "All birds can fly."**

*   **Predicate Logic:** ∀x (Bird(x) → CanFly(x))
    *   ∀ (for all): universal quantifier
    *   Bird(x): predicate asserting that x is a bird
    *   CanFly(x): predicate asserting that x can fly
    *   → (implies): logical implication

*   **Back to English:** "For every entity x, if x is a bird, then x can fly." or simply "If something is a bird, then it can fly."


**Constraint 2: "Some cars are red."**

*   **Predicate Logic:** ∃x (Car(x) ∧ IsRed(x))
    *   ∃ (there exists): existential quantifier
    *   Car(x): predicate asserting that x is a car
    *   IsRed(x): predicate asserting that x is red
    *   ∧ (and): logical conjunction

*   **Back to English:** "There exists at least one entity x such that x is a car and x is red." or "There is at least one car that is red."